In [46]:
from __future__ import division, print_function
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from dataloader import get_data
from sklearn.model_selection import train_test_split

In [3]:
tf.test.is_gpu_available()

True

In [4]:
X, y = get_data('../test_data/oscNext_genie_level3_v01.03_pass2.140000.000001.hdf5')

max pulses =  74


In [48]:
N_events, N_channels, _, N_features = X.shape
_, N_labels = y.shape

In [49]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

In [50]:
model = tf.keras.Sequential()

model.add(tf.keras.Input(shape=(N_channels, None, N_features))) #, ragged=True))
model.add(tf.keras.layers.TimeDistributed(tf.keras.layers.LSTM(16, input_shape=(None, N_features)), input_shape=(N_channels, None, N_features)))
model.add(tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(8), input_shape=(N_channels, 16)))
model.add(tf.keras.layers.Reshape((86, 60 * 8)))
model.add(tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(32, input_shape=(60*8,)), input_shape=(86,60*8)))
model.add(tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(32, input_shape=(100,)), input_shape=(86,100)))
model.add(tf.keras.layers.Flatten(input_shape=(86, 32)))
model.add(tf.keras.layers.Dense(32))
model.add(tf.keras.layers.Dense(32))
model.add(tf.keras.layers.Dense(N_labels))

model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_8 (TimeDist (None, 5160, 16)          1216      
_________________________________________________________________
time_distributed_9 (TimeDist (None, 5160, 8)           136       
_________________________________________________________________
reshape_2 (Reshape)          (None, 86, 480)           0         
_________________________________________________________________
time_distributed_10 (TimeDis (None, 86, 32)            15392     
_________________________________________________________________
time_distributed_11 (TimeDis (None, 86, 32)            1056      
_________________________________________________________________
flatten_2 (Flatten)          (None, 2752)              0         
_________________________________________________________________
dense_15 (Dense)             (None, 32)               

In [51]:
def angular_dist_loss(y_true, y_pred):
    return tf.reduce_mean(tf.math.square(tf.math.acos(tf.math.cos(y_pred[0]) * tf.math.cos(y_true[0]) + tf.math.sin(y_pred[0]) * tf.math.sin(y_true[0]) * tf.math.cos(y_pred[1] - y_true[1]))), axis=-1)

In [52]:
model.compile(loss=angular_dist_loss, optimizer="adam", )

In [54]:
model.fit(X_train, y_train, batch_size=32, epochs=1, validation_data=(X_test, y_test), )

Train on 11901 samples, validate on 1323 samples
11901/11901 [==============================] - 86s 7ms/sample - loss: 2.0242 - val_loss: 2.1432


In [33]:
y_pred = model.predict(X)

In [40]:
angular_dist_loss(y, y_pred)

<tf.Tensor: id=6940, shape=(), dtype=float32, numpy=1.8536657>

In [45]:
angular_dist_loss(y[::-1], y) * 2

<tf.Tensor: id=6996, shape=(), dtype=float32, numpy=3.1560388>